In [1]:
from twinyn.agents.seed_agent import *
from twinyn.agents.agent_functions import execute_sql
from twinyn.agents.prompts import *

In [2]:
from pathlib import Path

from autogen import ChatResult
from autogen.coding import LocalCommandLineCodeExecutor

import os
from dotenv import load_dotenv
load_dotenv()

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

In [3]:
executor = LocalCommandLineCodeExecutor(work_dir=work_dir, functions=[execute_sql])
seed_prompt = "Peak traffic time in the past 5 hours bucketed in 30 minute intervals"

In [4]:
llm_config_sql = {"config_list": [{"model": "gpt-4o-2024-08-06", "api_key": os.getenv("OPENAI_API_KEY"), "price": [0.0025, 0.0100]}]}
llm_config_analyst = {"config_list": [{"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY"), "price": [0.000150, 0.0006]}]}

In [5]:
seed_prompts = [
    "Peak traffic time in the past 5 hours bucketed in 30 minute intervals",
    "Request distribution by country in the past 5 hours"
]

In [6]:
conversations: list[ChatResult] = []
outputs: list[SeedOutput] = []

In [7]:
for s in seed_prompts:
    conversations.append(
        SeedTask(executor, llm_config_sql, llm_config_analyst, s, SQL_AGENT_SYSTEM_PROMPT, ANALYST_AGENT_SYSTEM_PROMPT)
    )
    conversations[-1].kickoff()
    
    outputs.append(
        SeedOutput(conversations[-1].chat_result)
    )
    outputs[-1].collect()


********************************************************************************
Starting a new chat....

********************************************************************************
code_executor_agent (to sql_agent):

Peak traffic time in the past 5 hours bucketed in 30 minute intervals

--------------------------------------------------------------------------------
sql_agent (to code_executor_agent):

To determine the peak traffic time in the past 5 hours, bucketed in 30-minute intervals, we need to follow these steps:

1. Calculate the last request time from the `server_log` table.
2. Define a time window of the last 5 hours based on this last request time.
3. Group the requests within this period into 30-minute intervals.
4. Count the number of requests in each 30-minute interval.
5. Identify the interval with the maximum number of requests.

Let's write the SQL query to achieve this and use the function `execute_sql` to execute the query and print the results. 

```python
f